# **Hands-on Motion Estimation and Correction**
*by [Gastao Cruz](https://twitter.com/gastaocruz) and [Thomas Kuestner](https://twitter.com/kuestnerthomas)*

In this hands-on of the [ISMRM Workshop on Motion Detection and Correction 2022](https://www.ismrm.org/workshops/2022/Motion/), we aim to convey the principles of motion artifacts, their appearance in the MR image, means of estimating motion (conventional and deep learning) and correcting for the induced motion artifacts during reconstruction.

This notebook is based on a [Github repository](https://github.com/lab-midas/ismrm-moco-workshop).

## Getting started
First you need to run the 
- **[0. Prerequisites](#Prerequisites)** which [installs the Python packages and libraries](#Installation), [imports the modules](#Imports) and [loads the data](#DataLoading) (see Provided material). In this hands-on, we will only focus on 2D single-slice brain MR image. If you want to try out the other datasets, you may need to adapt some functions, as only 2D processing is supported.

Afterwards you can decide to run either (or all) of the following three topics (in any order):
- **[1. Motion artifact appearance](#MotionArtifactAppearance)**: Learn something about the spatial and temporal behaviour of motion and how it manifests in the image for Cartesian and non-Cartesian imaging
- **[2. Motion estimation / image registration](#MotionEstimation)**: Learn how to perform an image registration using conventional and deep learning based solutions
- **[3. Motion-compensated image reconstruction](#MotionCompensated)**: Learn to incorporate motion estimation into image reconstruction to obtain a motion-corrected image

## Provided material
Two datasets (brain and heart) are distributed with this hands-on tutorial:
- `brain_large.npz` (GIT LFS): multi-coil 3D T1w brain MR image; matrix size = 256 x 216 x 65 x 7 (X x Y x Slices x Channels)
- `brain_slice.npz`: single central slice of `brain_large.npz`; matrix size = 256 x 216 x 7 (X x Y x Channels)
- `heart_large.npz` (GIT LFS): multi-coil, multi-slice 2D cardiac CINE MR image; matrix size = 176 x 132 x 12 x 25 x 10 (X x Y x Slices x Cardiac Phases x Channels)

**Attention:** The following codes are only defined for 2D processing. If you plan to use them for a 3D dataset, you would first need to extend some of the functions below.

# 0. Prerequisites
<a name="Prerequisites"></a>

**Attention:** If you are running this code in Google Colab, at the end of the installation you should receive a message that the kernel has crashed and restarted. This kernel restart is required so that all installed libraries work correctly.

Afterwards you can continue with [Imports](#Imports).

In [ ]:
%%bash
export USE_OPTOX=false  # [false | true]: optox is only needed if you want to work with the MERLIN provided functions for conjugate gradient etc.

## Installation
<a name="Installation"></a>

In [ ]:
%%bash
# apt-get install doxygen

### Python packages

In [ ]:
%%bash
# package itself
if [ -d "/content/ismrm-moco-workshop" ]; then
  cd /content/ismrm-moco-workshop
  git pull
else
  git clone https://github.com/lab-midas/ismrm-moco-workshop.git
fi
cp -r /content/ismrm-moco-workshop/utils /content/

In [ ]:
# install dependencies
! pip install --upgrade pip
! pip install simpleitk git+https://github.com/khammernik/medutils.git python-pysap scikit-image voxelmorph # imjoy 
! git clone https://github.com/voxelmorph/voxelmorph.git

In [ ]:
# Download ESPIRiT code for coil sensitivity map estimation
!git clone https://github.com/mikgroup/espirit-python.git
!cp /content/espirit-python/espirit.py .

### GPUNUFFT

In [ ]:
%%bash
# setup gpunufft
git clone --branch cuda_streams https://github.com/khammernik/gpuNUFFT.git
export CUDA_ROOT_DIR=/usr/local/cuda
#!export CUDA_SDK_ROOT_DIR=/usr/local/cuda-samples
export LDFLAGS=-L/usr/local/cuda/lib64
mkdir -p gpuNUFFT/CUDA/build
cd gpuNUFFT/CUDA/build
cmake .. -DGEN_MEX_FILES=OFF
make

In [ ]:
%%bash
apt install libnfft3-dev
pip install gpuNUFFT pynfft2

### Optox

In [ ]:
# Ensure only tensorflow with GPU support is installed
! pip uninstall --yes tensorflow
! pip install tensorflow-gpu==2.8  # bug in cuDNN 8 + TF2.9 for Colab (crashes voxelmorph)

In [ ]:
#!pip install tensorflow-gpu==2.8
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

In [ ]:
# set some environment variables in Google Colab that are not automatically handled
import os
import tensorflow as tf

os.environ["Tensorflow_INCLUDE_DIRS"] = str(tf.sysconfig.get_include())
os.environ["Tensorflow_LIB_DIRS"] = str(tf.sysconfig.get_lib())
os.environ["Tensorflow_LIB"] = str(tf.sysconfig.get_link_flags()[1])
os.environ["Tensorflow_FLAGS"] = str(tf.sysconfig.get_compile_flags()[1])

In [ ]:
%%bash
# setup optox
git clone https://github.com/midas-tum/optox.git
export GPUNUFFT_ROOT_DIR=/content/gpuNUFFT
export LDFLAGS=-L/usr/local/cuda/lib64
cd /content/optox
pip install -r requirements.txt
mkdir -p /content/optox/build
cd /content/optox/build
cmake .. -DWITH_PYTHON=ON -DWITH_PYTORCH=OFF -DWITH_TENSORFLOW=ON -DCUDA_SDK_ROOT_DIR=/usr/local/cuda/samples -DCUDA_ROOT=/usr/local/cuda/
make install

### MERLIN

In [ ]:
%%bash
# setup MERLIN
git clone https://github.com/midas-tum/merlin.git
cd merlin/python
python ./setup.py build
cd ..
pip install ./python
pip install ./pytorch
pip install ./tensorflow

### BART



We install dependencies for BART. Make sure that you have installed the requirements if you run locally:

In [ ]:
%%bash

[ $COLAB -ne 1 ] && echo "Skipp cell (not on Colab)" && exit 0

# Install BARTs dependencies
apt-get install -y make gcc libfftw3-dev liblapacke-dev libpng-dev libopenblas-dev &> /dev/null

# Install additional dependencies for converting ISMRMRD files
apt-get install -y libismrmrd-dev libboost-all-dev libhdf5-serial-dev &> /dev/null



#### Clone and Compile BART

We clone BART into the current working directory of this notebook and delete any previous installation in this directory.

In [ ]:
# Clone Bart
! [ -d bart ] && rm -r bart
! git clone https://github.com/mrirecon/bart/ bart &> /dev/null

In [ ]:
%%bash

cd bart

# Define compile options
COMPILE_SPECS=" PARALLEL=1
                CUDA=1
                NON_DETERMINISTIC=1
                ISMRMRD=1
                "

printf "%s\n" $COMPILE_SPECS > Makefiles/Makefile.local


#if [ $COLAB -eq "1" ];
#then
# set path to cuda for Colab
echo "CUDA_BASE=/usr/local/cuda" >> Makefiles/Makefile.local
echo "CUDA_LIB=lib64" >> Makefiles/Makefile.local
#fi


make &> /dev/null

#### Add BART to PATH variable

We add the BART directory to the PATH variable and include the python wrapper for reading *.cfl files:

In [ ]:
import os
import sys

os.environ['TOOLBOX_PATH']=os.getcwd()+"/bart/"
os.environ['PATH'] = os.environ['TOOLBOX_PATH'] + ":" + os.environ['PATH']
sys.path.append(os.environ['TOOLBOX_PATH'] + "/python/")

Check BART setup:

In [ ]:
%%bash

echo "# The BART used in this notebook:"
which bart
echo "# BART version: "
bart version

Some helper functions:

In [ ]:
%matplotlib inline

def plotcfl(files, title=None):
  import numpy as np
  from matplotlib import pyplot as plt
  import cfl
  import os

  from ipywidgets import interact, interactive, fixed, interact_manual
  import ipywidgets as widgets

  def update(Range=(0.,1.),Coil=0, Map=0, Slice=0, Batch=0):

    if(title==None):
      wtitle=[]
      for file in files:
        head, tail = os.path.split(file)
        wtitle.append(tail)
    else:
      wtitle=title
    
    data=cfl.readcfl(files[0])
    rat=data.shape[0]/data.shape[1]
    width=16
    
    ncols=len(files)
    nrows=1
    
    rat=rat
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, constrained_layout=True, squeeze=False, figsize=(width, width*rat))

    for i in range(len(files)):
 
        data=cfl.readcfl(files[i])
        nshp = [1] * 16

        for k in range(len(data.shape)):
          nshp[k] = data.shape[k]

        idx=[0]*16
        idx[3]=min(Coil,nshp[3]-1)
        idx[4]=min(Map,nshp[4]-1)
        idx[13]=min(Slice,nshp[13]-1)
        idx[15]=min(Batch,nshp[15]-1)

        idx[0]=slice(None, None, -1)
        idx[1]=slice(None, None, None)

        dat=np.abs(data.reshape(nshp)[tuple(idx)])
        if 0 < np.max(dat):
          dat=dat/np.max(dat)

        axs.flatten()[i].imshow(dat,cmap="gray",vmin=Range[0], vmax=Range[1])
        axs.flatten()[i].set_title(wtitle[i])
    for ax in axs.flatten():
      ax.axis("off")
    
    plt.show()

  nshp = [1] * 16
  for file in files:
    data=cfl.readcfl(file)
    for i in range(len(data.shape)):
      nshp[i] = max(data.shape[i], nshp[i])
  
  interact(update,
           Range=widgets.FloatRangeSlider(value=[0.,1.], min=0, max=1.),
           Coil=widgets.IntSlider(min=0, max=nshp[3]-1, step=1, value=0),
           Map=widgets.IntSlider(min=0, max=nshp[4]-1, step=1, value=0),
           Slice=widgets.IntSlider(min=0, max=nshp[13]-1, step=1, value=0),
           Batch=widgets.IntSlider(min=0, max=nshp[15]-1, step=1, value=0))

In [ ]:
%%bash
function imshow () {
    bart toimg -W $1 $1.png > /dev/null
    cat $1.png | display
    rm $1.png
}
export -f imshow

# version without automatic windowing
function imshow_nowin () {
    bart toimg $1 $1.png > /dev/null
    cat $1.png | display
    rm $1.png
}
export -f imshow_nowin

### Matplotlib

In [ ]:
%%bash
pip install matplotlib==3.2.2  # correct plotting bug

### Final checks

In [ ]:
print("Restarting of kernel...")  # needed to load installed libraries
get_ipython().kernel.do_shutdown(True)

In [ ]:
# inspect the available GPU hardware
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!nvidia-smi

In [ ]:
pip list

## Imports
<a name="Imports"></a>

**ATTENTION:** Please make sure that you have restarted the Jupyter kernel after installation of the above libraries (error message of unexpected crash and restart). Otherwise, the installed libraries will not be found. You can restart the kernel with:

```
get_ipython().kernel.do_shutdown(True)
```



In [ ]:
import os 
import sys

os.environ['TOOLBOX_PATH']=os.getcwd()+"/bart/"
os.environ['PATH'] = os.environ['TOOLBOX_PATH'] + ":" + os.environ['PATH']
sys.path.append(os.environ['TOOLBOX_PATH'] + "/python/")

import numpy as np
import medutils 
import SimpleITK as sitk
import matplotlib.pyplot as plt

try:
  import optotf
except:
  print('Optox not installed')
import tensorflow as tf
import espirit

import voxelmorph as vxm
import neurite as ne

# own utils
from utils.flowplotting import *
from utils.imageplotting import *
from utils.mri import *
from utils.padding import *
from utils.radialsampling import *
from utils.cartesiansampling import *
from utils.warping import *
from utils.motionsim import *
from utils.motioncomp import *

from mri.operators import NonCartesianFFT

## Data loading
<a name="DataLoading"></a>

In [ ]:
# Load the numpy
import numpy as np
from utils.padding import zpad 

datapath = '/content/ismrm-moco-workshop/data/brain_slice.npz'  # brain_slice 
data = np.load(datapath)
img = data['arr_0']
nRead = np.amax(np.shape(img)[0:2])
img = zpad(img, (nRead, nRead, np.shape(img)[2])).astype(np.complex64)  # make image quadratic for 

nX, nY, nCha = np.shape(img)

In [ ]:
plot(img, title='Coil images')

# 1. Motion artifact appearance 
<a name="MotionArtifactAppearance"></a>
In this part, we will examine the impact of the motion artifacts on the image. Motion during an MR scan introduces blurring and aliasing (along the phase-encoding directions). Artifact appereance depends on the imaging trajectory: Cartesian and non-Cartesian. In the following we will first investigate the motion artifact appeareance on fully-sampled cases and then move towards undersampled/accelerated imaging trajectories.



## **Spatial description of motion**
For translational, rigid and affine motion exists a direct linear relationship between the motion-corrected/free k-space $\nu(k)$ and the motion-affected k-space $\nu'(k')$.

$$
\nu(k) = \nu'(k') \frac{\operatorname{exp}(2\pi i k'\cdot t)}{|\det (A)|}
$$
where $k'$ is the motion-affected k-space trajectory, $k$ is the motion-corrected/free k-space trajectory, $t=\lbrack t_x, t_y, t_z \rbrack^T$ is the translational component of the motion and $A$ is the affine motion matrix in 3D

$
\begin{align}
A = RGS = R_x R_y R_z G S = \begin{bmatrix} 1 & 0 & 0 \\ 0 & \cos \phi & -\sin \phi \\ 0 & \sin \phi & \cos \phi \end{bmatrix} \begin{bmatrix} \cos \theta & 0 & \sin \theta \\ 0 & 1 & 0 \\ -\sin \theta & 0 & \cos \theta \end{bmatrix} \begin{bmatrix} \cos \psi & -\sin \psi & 0 \\ \sin \psi & \cos \psi & 0 \\ 0 & 0 & 1 \end{bmatrix} \begin{bmatrix} 1 & G_{xy} & G_{xz} \\ 0 & 1 & G_{yz} \\ 0 & 0 & 1 \end{bmatrix} \begin{bmatrix} S_x & 0 & 0 \\ 0 & S_y & 0 \\ 0 & 0 & S_z \end{bmatrix}
\end{align}
$

consisting of the rotation matrices $R_x, R_y$ and $R_z$ (with angles $\lbrace \phi, \theta, \psi \rbrace$) along the respective axis $x, y$ and $z$, shearing matrix $G$ (with 3D shear parameters $\lbrace G_{xy}, G_{xz}, G_{yz} \rbrace$) and scaling matrix $S$ (with 3D scale factors $\lbrace S_x, S_y, S_z \rbrace$). 

In 2D, the affine motion matrix simplifies to

$
A=RGS = \begin{bmatrix} \cos \phi & \sin \phi \\-\sin \phi & \cos \phi \end{bmatrix} \begin{bmatrix} 1 & G_{xy} \\ 0 & 1 \end{bmatrix} \begin{bmatrix} S_x & 0 \\ 0 & S_y \end{bmatrix}
$

### Affine motion
The 3D affine transformation has 12 degrees of freedom consisting of 3D translations, rotations, shears and scaling. It maps the k-space point $k=\lbrack k_x, k_y, k_z \rbrack^T$ to the corrupted k-space point $k'=\lbrack k'_x, k'_y, k'_z \rbrack^T$ following $k'=A^{-T} k$ 

in k-space: $
\begin{bmatrix} k'_x \\ k'_y \\ k'_z \end{bmatrix} = R \begin{bmatrix} 1 & 0 & 0 \\ -G_{xy} & 1 & 0 \\ (G_{xy}G_{yz}-G_{xz}) & -G_{yz} & 1\end{bmatrix} \begin{bmatrix} S_x^{-1} & 0 & 0 \\ 0 & S_y^{-1} & 0 \\ 0 & 0 & S_z^{-1} \end{bmatrix} \begin{bmatrix} k_x \\ k_y \\ k_z \end{bmatrix}
$

or equivalently 

in image domain: $
\begin{bmatrix} x' \\ y' \\ z' \\ 1 \end{bmatrix} = \begin{bmatrix} & & & t_x \\ & R & & t_y \\ & & & t_z \\ 0 & 0 & 0 & 1 \end{bmatrix} GS \begin{bmatrix} x \\ y \\ z \\ 1 \end{bmatrix}
$

In this case, we are considering the augmented matrix form (i.e. 4D for 3D spatial), which conveniently allows us to write translations and the other affine components as a series of matrix multiplications.

### Rigid motion
In case of rigid motion, we only have translation $t$ and rotation $R$, i.e. $G=S=I$, where $I$ is the identity matrix.

### Translational motion
For translational motion $t$, all other components are neglected, i.e. $R=G=S=I$.

### Elastic (non-rigid) motion 
For elastic motion, the image $\rho \in \mathbb{R}^N$ is deformed with a deformation field/motion model $u$ which is a vector field containing the motion vector at every voxel $u \in \mathbb{R}^{N \times D}$ for a $D$-dimensional registration. The deformed image is given as
$
\rho_d(x') = \rho(u(x))
$.

## **Temporal description of motion**
Motion represents a displacement over time and this can be addressed most directly by stating the spatial variation (i.e. the affine motion parameters) as a function of time. In practice, however, this belies the complexity of real physiologic motion, where limits on real-world acceleration, shear, etc. limit how displacement can practically vary over time. Moreover, some choice of discretization for the time domain is necessary.

### Regular motion
Many types of physiologic motion exhibit temporal regularity. Respiration has an inspiration and expiration phase; the heartbeat can be characterized by an approximately repeating waveform, identifiable by the QRS complex in the ECG. A key aspect of these types of motion is that while they are generally periodic, they do not necessarily have a fixed frequency through the duration of a scan — a subject may hold their breath; a heartbeat may skip. Therefore, these types of motion are often described as cyclic rather than periodic.

### Irregular motion
Irregular motions (e.g., head motion) are generally represented using a discretized time series of serial displacements.

## Motion in Cartesian imaging
We will first examine the impact of motion on Cartesian imaging. We can apply the motion in the image domain by the function: 

```
def transform_img(img, p): 
# img      input image to be transformed
# p        affine transformation parameters
#          3D (rank(img) == 3): t_x, t_y, t_z, \phi [°], \theta [°], \psi [°], G_{xy}, G_{xz}, G_{yz}, S_x, S_y, S_z
#          2D (rank(img) == 2): t_x, t_y, \phi [°], G_{xy}, S_x, S_y
# return   transformed image
```
takes care of creating and applying the transformation to the input `img`. 



### Motion in fully-sampled imaging
**Task 1:** Apply a 2D translational motion with $t_x=10, t_y=5$ to the RSS coil-combined image (`img_rss = rss(img)`) and plot the original and transformed image side-by-side using the function `plot([img_a, img_b])`. 


```
def plot(img, flow=None, ...):
# img        to be plotted image or list of 2D images
#            2D: single 2D image
#            3D: third dimension is plotted side-by-side
# flow       2D: [x, y, flowDir]
#            3D: [x, y, slices, flowDir]
#            flowDir being the x and y components of the flow
```



In [ ]:
# TODO

**Task 2:** Apply an in-plane rotation of $\phi=30^\circ$. 

In [ ]:
# TODO

**Task 3:** Try different combinations of translation, rotation, scalings and shearings. 

In [ ]:
# TODO

**Task 4:** Write the equivalent translational operation of the image domain transformation in k-space. Plot the translational offsetted image performed in image domain, k-space domain and the absolute error between image domain and k-space domain transformation.<br/>
*Optional:* Write the equivalent affine operation in k-space.

In [ ]:
# TODO

Let us now simulate a simple case of a motion-affected MR scan. Therefore, we will impact every $n$-th phase-encoding line with motion using the function `simulate_motion()`. In order to simulate it, we first need to set up the functions for a zero-filled reconstruction.

### Zero-filled reconstruction
In order to simulate motion and ultimately reconstruct an image, we need the multi-coil forward operator $A$ in `mriForwardOp(image, mask, smaps)` and adjoint operator $A^∗$ in `mriAdjointOp(kspace, mask, smaps)`, as well as an estimation of the coil sensitivities.

*Hint*: We perform a centered FFT with `ortho` normalization in order to provide adjoint operators. 
Suggested Readings:<br/>
Pruessmann et al. [SENSE: Sensitivity encoding for fast MRI](https://onlinelibrary.wiley.com/doi/abs/10.1002/%28SICI%291522-2594%28199911%2942%3A5%3C952%3A%3AAID-MRM16%3E3.0.CO%3B2-S) Magnetic Resonance in Medicine, 43(5):952-962, 1999.

#### Forward and adjoint operator
The operators are already defined (and imported) in `utils/mri.py`:


```
def mriAdjointOp(kspace, mask, smaps):
  return np.sum(ifft2c(kspace * mask)*np.conj(smaps), axis=-1)

def mriForwardOp(image, mask, smaps):
  return fft2c(smaps * image[:,:,np.newaxis]) * mask

def fft2c(image, axes=(0,1)):
  return np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(image, axes=axes), norm='ortho', axes=axes), axes=axes)

def ifft2c(kspace, axes=(0,1)):
  return np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(kspace, axes=axes), norm='ortho', axes=axes), axes=axes)
```


#### Coil sensitivity estimation
The coil sensitivity maps (`smaps`) are smooth maps that show us in which parts the individual coil elements are sensitive. We will need these information for our multi-coil MRI forward and adjoint operators. We use the [python implementation](https://github.com/mikgroup/espirit-python) for ESPIRiT [1,2] to estimate these coil sensitivity maps.

[1] Uecker et al. [ESPIRiT—an eigenvalue approach to autocalibrating parallel MRI: Where SENSE meets GRAPPA](https://onlinelibrary.wiley.com/doi/10.1002/mrm.24751). Magn Reson Med 71(3):990-1001, 2014.

[2] https://github.com/mikgroup/espirit-python

In [ ]:
import espirit
kspace = fft2c(img)
kspace_espirit = kspace[:,:,np.newaxis,:]
smaps_espirit = espirit.espirit(kspace_espirit, 8, 20, 0.05, 0)

smaps = smaps_espirit[:,:,0,:,0]

Let us visualize the sensitivity maps

In [ ]:
plot(smaps)

Now, you should check if the adjoint operator is working as expected. The result should be a coil-combined image. Right now, there is no undersampling mask involved, i.e., it is set to all ones.

In [ ]:
sampling_mask_fs = np.ones_like(kspace)  # fully-sampled sampling mask
img_cc = mriAdjointOp(kspace, sampling_mask_fs, smaps)
plot(img_cc)

### Motion simulation
Now we can simulate motion in the acquired k-space. The temporal behaviour, i.e. when the motion happens can be described by motion signals, which we will look into more detail in the next section. First, we will simulate an abrupt and periodic motion. We will make use of the function `simulate_motion()` in `utils/motionsim.py`. 


```
def simulate_motion(img_cc, smaps, mask, p):
  # img_cc      motion-free coil-combined image
  # smaps       coil sensitivity maps
  # mask        k-space sampling mask 
  # p           a) affine motion parameters (constant over time), 1x6
  #             b) affine motion course (time-dependent or time-constant), nPE x 6
  #             np.abs(p[:, :5]) > 0 = mask_motion; time points of phase-encoding steps 
  #             at which motion parameters > 0 are defined, i.e. motion is happening 
  # return:     motion-affected k-space, motion mask
```



**Task 5:** Simulate a motion-affected image, such that every 4-th phase-encoding line (i.e. abrupt motion) is affected by a translational motion with $t_x=10, t_y=5$.

In [ ]:
# p = TODO 

kspace_motion, mask_motion = simulate_motion(img_cc, smaps, sampling_mask_fs, p)
img_fs_const_motion = mriAdjointOp(kspace_motion, sampling_mask_fs, smaps)

**Task 6:** Let us now visualize the motion-free and the motion-affected image side-by-side.

In [ ]:
# TODO

### Temporal variation of motion
Motion represents a displacement over time and this can be addressed most directly by stating the spatial variation (i.e. the affine motion parameters) as a function of time. 

The simulated motion from the previous example affected every $n$-th phase-encoding line, i.e. every $n$ TRs (assuming one echo line per TR) an abrupt motion was happening. We can depict the temporal behaviour of the motion as a function of affine motion parameters over time with the help of `plot_motion_course()` in `utils/motionsim.py`. 


```
def plot_motion_course(motion_course, TR=1):
  # motion_course     temporal variation of motion, i.e. temporal course of motion parameters, shape: [motion_parameters, total_time]
  # TR                repetition time [ms]
```



**Task 7:** Plot the affine motion parameters for an abrupt translational motion ($t_x=10, t_y=5$) as a function over time for a TR=5 ms and one echo (phase-encoding line) per TR.

In [ ]:
p = [10, 5, 0, 0, 1, 1]
TR = 5E-3

# abrupt motion
motion_course = np.zeros((np.shape(mask_motion)[1], len(p)))
# TODO

plot_motion_course(motion_course, TR)

**Task 8:** Create a periodic motion whose translational motion $t_x$ follows a sinus rhythm with frequency $\omega_x=5 \text{s}^{-1}$ , maximal amplitude 15, and $t_y$ follows a cosine rhythm with a frequency $\omega_y=6 \text{s}^{-1}$, maximal amplitude 8. Both motions happen at every 4-th phase-encoding line. Plot the affine motion parameters over time (TR=5 ms).

In [ ]:
p = [15, 8, 0, 0, 1, 1]
TR = 5E-3

# periodic motion
# TODO

plot_motion_course(motion_course, TR)

**Task 9:** Corrupt the k-space now with the time-dependent motion parameters from **Task 8** and visualize the motion-affected image.

In [ ]:
# TODO

It is also conceivable that different motion parameters are happening at different time points, i.e. we will have a different `mask_motion` for each motion parameter.

**Task 10:** Create a periodic motion whose translational motion $t_x$ follows a sinus rhythm with frequency $\omega_x=5 \text{s}^{-1}$ , maximal amplitude 15, and $t_y$ follows a cosine rhythm with a frequency $\omega_y=6 \text{s}^{-1}$, maximal amplitude 8. Translation $t_x$ happens every 4-th phase-encoding line, whereas translation $t_y$ happens at every 8-th phase-encoding line. Plot the affine motion parameters over time (TR=5 ms).

In [ ]:
p = [15, 8, 0, 0, 1, 1]
TR = 5E-3

# periodic motion
# TODO

plot_motion_course(motion_course, TR)

### Motion in undersampled/accelerated imaging
**Task 11:** Perform a Cartesian undersampling (regular and random) using the provided function `generate_mask()` from `utils/cartesiansampling.py`:


```
def generate_mask(R, nPE, nFE, nRef=20, mode='regular'):
  # R     desired acceleration factor
  # nPE   amount of phase-encoding lines
  # nFE   amount of frequency-encoding lines
  # nRef  amount of fully-sampled center lines
  # mode  'regular': Parallel-Imaging-like/regular undersampling, 'random': Compressed-Sensing-like/random undersampling
```




Now, generate a regular undersampling mask and visualize it (we visualize only a fraction in frequency encoding direction).



In [ ]:
np.random.seed(1001)
sampling_mask = generate_mask(R=3, nPE=np.shape(img_cc)[1], nFE=np.shape(img_cc)[0], mode='regular')

medutils.visualization.imshow(sampling_mask[:40,:,0], 'Undersampling mask', figsize=(20,20))

Next you can examine the impact of the undersampling masks on the reconstructed image

**Task 12:** Examine the impact of a `regular` undersampling on the image for changing acceleration factors.

In [ ]:
# TODO

**Task 13:** Examine the impact of a `random` undersampling on the image for changing acceleration factors.

In [ ]:
# TODO

Let us now combine accelerated imaging with the impact of motion.

**Task 14:** Induce motion in the `regular` undersampled data, assuming every 4-th phase-encoding line is affected by translational motion $t_x=10, t_y=5$. Plot the motion-free, fully-sampled motion-affected, and undersampled motion-affected image side-by-side.

In [ ]:
# TODO

**Task 15:** Induce motion in the `random` undersampled data, assuming every 4-th phase-encoding line is affected by translational motion $t_x=10, t_y=5$. Plot the motion-free, fully-sampled motion-affected, and undersampled motion-affected image side-by-side.

In [ ]:
# TODO

## Motion in radial imaging
Let's examine the impact of motion in radial imaging. We will therefore first need to radially sample the data. For reconstruction, we use the [gpuNUFFT](https://github.com/andyschwarzl/gpuNUFFT) package with the Python interface provided by [pySAP-mri](https://github.com/CEA-COSMIC/pysap-mri/). Alternatively, one could also use the [BART](https://mrirecon.github.io/bart/) toolbox.

For simplicity, we first zero-pad the image to a quadrativ field of view. Next, we recalculate the coil sensitivity maps.

In [ ]:
import espirit

# getting the image into quadrativ FOV
#img_qfov = np.pad(img, ((0,), (int(np.abs(nX-nY)/2),), (0,)))
nRead = np.amax(np.shape(img)[0:2])
img_qfov = zpad(img, (nRead, nRead, np.shape(img)[2])).astype(np.complex64)
img_qfov = img_qfov[:,:,np.newaxis,:]
kspace_qfov = fft2c(img_qfov)

smaps_rad_espirit = espirit.espirit(kspace_qfov, 8, 20, 0.05, 0)
smaps_rad = smaps_rad_espirit[:,:,0,:,0]

### Option A) gpuNUFFT


We can use the [gpuNUFFT](https://github.com/andyschwarzl/gpuNUFFT) package to reconstruct an image from a retrospective radial sampling. Please see **Task 16** for more information.

### Option B) BART

We can use the [BART](https://mrirecon.github.io/bart/) toolbox to perform a radial sampling and NUFFT reconstruction.

In [ ]:
import cfl
cfl.writecfl('imgrad', img_qfov)

In [ ]:
%%bash
SAMPLES=256
SPOKES=402  # pi/2 * SAMPLES
bart traj -x$SAMPLES -y$SPOKES -r tmp_traj
bart scale 0.5 tmp_traj traj
bart nufft traj imgrad kspace_rad
bart show -m kspace_rad
bart nufft -a traj kspace_rad imgreco

In [ ]:
plotcfl(['imgreco'])
imgreco = cfl.readcfl('imgreco')
plot(imgreco)

### Motion in fully-sampled and undersampled/accelerated imaging

**Task 16:** Define the non-uniform FFT (NUFFT) operator `NonCartesianFFT(samples, shape, n_coils, density_comp, smaps, implementation='gpuNUFFT')` using the [gpuNUFFT](https://github.com/andyschwarzl/gpuNUFFT) package and which requires the following inputs:
- `samples`: radial sampling trajectory (shape: `[nSpokes, 2]`)
- `shape`: (list) of quadrative field of view size: `[nRead, nRead]`
- `n_coils`: amount of MR coils
- `density_comp`: density compensation function (shape: `[nSpokes, 1]`)
- `smaps`: coil sensitivity maps (shape: `[coils, nRead, nRead]`)

Plot the radially sampled image.

*Hint:* You can use the function `prepare_radial()` in `utils/radialsampling.py` to obtain the radial sampling trajectory and the density compensation function:

```
def prepare_radial(acc, nRead, nSlices=1):
"""
acc:         acceleration factor
nRead:       number of readout steps
nSlices:     number of slices
:return:     radial trajectory, density compensation function
"""
```

In [ ]:
from mri.operators.utils import convert_locations_to_mask
from mri.operators import NonCartesianFFT

img_rad = rss(img_qfov[:,:,0,:])
csm = np.transpose(smaps_rad, (2, 0, 1))
kpos, dcf = prepare_radial(acc=1, nRead=nRead)

# NUFFT operator
# TODO

**Task 17:** Perform a retrospectively accelerated radial imaging with various acceleration factors and plot the image.

In [ ]:
kpos, dcf = prepare_radial(acc=8, nRead=nRead)

# TODO

**Task 18:** Similar to the Cartesian function `simulate_motion()` in `utils/motionsim.py` create a function that simulates the motion on a radially sampled k-space. 

In [ ]:
# TODO

**Task 19:** Simulate that every 4-th k-space radial readout is affected by a translational motion with $t_x=10, t_y=5$ for a fully-sampled acquisition (`acc=1`). Plot the motion-free and the motion-affected image.

In [ ]:
p = [10, 5, 0, 0, 1, 1]
kpos, dcf = prepare_radial(acc=1, nRead=nRead)

# TODO

**Task 20:** Plot and compare the fully-sampled (Cartesian/non-Cartesian), fully-sampled motion-affected Cartesian and fully-sampled motion-affected non-Cartesian (radial) images. Which one is less sensitive to motion?

In [ ]:
# TODO

**Task 21:** Simulate that every 4-th k-space radial readout is affected by a translational motion with $t_x=10, t_y=5$ for an accelerated acquisition (`acc=8`). Plot the motion-free and the motion-affected image.

In [ ]:
p = [10, 5, 0, 0, 1, 1]
kpos, dcf = prepare_radial(acc=8, nRead=nRead)

# TODO

**Task 22:** Repeat the above examples for varying affine motion parameters $p$, motion courses (constant or changing over time), motion time points ($n$-th readout) and acceleration factors of the radial undersampling. Plot and compare the images.

In [ ]:
p = [10, 5, 30, 0, 1, 1]
kpos, dcf = prepare_radial(acc=1, nRead=nRead)

# TODO

p = [10, 5, 20, 0.1, 1, 1]
# TODO

# 2. Motion estimation / image registration
<a name="MotionEstimation"></a>
In this part we will learn about algorithms to register and estimate the motion. We will illustrate conventional (optical flow, B-Spline) and deep-learning methods for this.

In this part of the tutorial we will have a closer look at image registration using conventional and deep learning-based solutions. We will only focus on pairwise registrations, i.e. registering a moving image $\rho_m(r)$ to a fixed image $\rho_f(r)$. The fixed and moving image are of dimension $N$ and are each defined on their own spatial domain: $\Omega_f \subset \mathbb{R}^N$ and $\Omega_m \subset \mathbb{R}^N$. The task of registration is to find a displacement $u(r)$ that makes $\rho_m(r+u(r))$ spatially aligned to $\rho_f(r)$. Equivalently we can say that registration is the process to find the transformation/displacement $T(r) = r+u(r)$ that makes $\rho_m(T(r))$ spatially aligned to $\rho_f(x)$. The transformation is defined as a mapping from the fixed image to the moving image, i.e. $T: \Omega_f \subset \mathbb{R}^N \to \Omega_m \subset \mathbb{R}^N$. The flow/displacement/motion field $u(x)$ contains the flows in the respective directions $u(r) = [u_x(r), u_y(r), \dots]$.

This transformation $T$, respectively its flow/displacement/motion field $u$ is also known as the *forward* motion $u^{(F)}$ (for simplicity we will refer to the forward motion field as $u$). Consequently, we also have a *backward* motion $u^{(B)}$. Ideally, the forward and backward motion should be isomorph, i.e. $u(u^{(B)}(r)) = r$ which allows to obtain either of the flows from the other one. For some cases, it might also be beneficial if the flows are diffeomorph, i.e. forward and backward motion are continously differentiable. During optimization, we are actually solving for the backward motion and later invert this flow to obtain the forward motion. This is usally more numerically stable as we expect that there is a matching voxel in the fixed image to which we can morph the moving image's voxel.   

The main properties of a registration algorithm are:
- dimensionality: 2D to 2D, 2D to 3D, 3D to 3D, ...
- spatial motion: rigid, affine, elastic (non-rigid)
- input modality: mono-modal, multi-modal
- similarity measure: landmark-based, image-intensity based
- cost function: mean-squared-error, mutual information, cross-correlation, ...
- regularization: pyramidal resolution, smoothness or energy constraints, Jacobian constraints, isomorph or diffeomorph constraints, ...
- algorithm: optical flow, diffusion-based, splines (B-splines, thin-plate splines, ...), deep learning, ...

## Conventional image registration
For the conventional image registration methods, we will have a look at the differences between optical flow and B-Spline based registrations, as these methods are the most commonly used ones. Moreover, deep learning-based registrations are often motivated or based on optical flow methods. 

### Optical Flow
We will first examine an optical flow-based image registration. Therefore we create a pair of motion-free and motion-affected image.

**Task 23:** Create a pair of motion-free (fixed) and moving image using `transform_img()` and `plot()` them side-by-side.

In [ ]:
# TODO
img_fixed = minmaxscale(img_fixed, [0, 1])
img_moving = minmaxscale(img_moving, [0, 1])
plot([img_fixed, img_moving])

**Task 24:** Let us define the optical flow registration using the scikit-image (or any other library of your liking) for the Lucas-Kanade algorithm. 

*Hint:* Check the orientation of the flow for the $x$ and $y$ component, i.e. verify the displacement for a translational motion (`[10, 0, 0, 0, 0, 0]`) in $x$ direction first.

In [ ]:
# TODO

**Task 25**: First stack the $x$ and $y$ components of the flow $u_x$, $u_y$ along the last dimension to obtain a flow field $u$ of shape $X \times Y \times 2$. Plot the obtained deformation field as quiver and color-plots using `plot` (in `utils/imageplotting.py`) and `plot_flow` (in `utils/flowplotting.py`), respectively.

For displaying the flow fields, we can show them as a quiver plot (vector field overlaid to image) using `plot([image1, image2], [flow_of_image1, flow_of_image2])` or as a color-encoded image (color depicts flow angle $\arg u$ and hue depicts magnitude $|u|$) using `plot_flow(flow)`.

The color-coded image has the following intepretation: <br/>
![](https://github.com/lab-midas/ismrm-moco-workshop/blob/master/doc/colorwheel.png?raw=true)

In [ ]:
flows = # TODO

# quiver plots
plot([img_fixed, img_moving], [np.zeros_like(flows), flows], scale=5, figsize=(20,20))

# color-wheel plot
plot_flow(flows)

**Task 26:** Inspect and display the deformed/warped image using `warp_2D()` in `utils/warping.py`.

In [ ]:
# TODO

**Task 27:** Perform the registration using the Total-variation L1-regularized optical flow algorithm.

In [ ]:
# TODO

**Task 28:** Repeat the **Tasks 25** and **26** with the TV L1-regularized optical flow from **Task 27**.

In [ ]:
# TODO

**Task 29:** Evaluate the obtained registration performance in the warped image (i.e. warped to fixed image) and compare against before registration (i.e. moving to fixed image). Compute the photometric loss = $\sum (\rho_\text{warped} - \rho_f)^2$, structural similarity (SSIM) and peak signal to noise ratio (PSNR). 

In [ ]:
# TODO

**Task 30:** Now we evaluate the performance of the registration on the obtained motion/displacement field. Calculate 
- the determinant of the Jacobian of the transformation <br/>
$\text{Jac} = \operatorname{det}(\nabla T(r)) = \left| \begin{bmatrix} \frac{\partial u_x}{\partial x} + 1 & \frac{\partial u_x}{\partial y} \\ \frac{\partial u_y}{\partial x} & \frac{\partial u_y}{\partial y} + 1 \end{bmatrix} \right|$
- the divergence of the displacement field <br/>
$\operatorname{div}(u) = \nabla \cdot u = \frac{\partial u_x}{\partial x} + \frac{\partial u_y}{\partial y}$
- absolute displacement field change in selected direction $p=\lbrace x,y \rbrace$ <br/>
$| \nabla u | = \sqrt{ \left(\frac{\partial u_x}{\partial p}\right)^2 + \left(\frac{\partial u_y}{\partial p}\right)^2}$

and plot the obtained metrics.

*Optional:* You may also try to implement these functions with SimpleITK ([1](https://itk.org/Doxygen/html/classitk_1_1DisplacementFieldJacobianDeterminantFilter.html), [2](https://simpleitk.org/doxygen/latest/html/classitk_1_1simple_1_1GradientImageFilter.html)). 

In [ ]:
# TODO

Depending on your simulated motion you can identify with these metrics, directional changes or contraction/expansion of the moved tissues.

**Task 31:** Invert the flow fields by changing the registration direction (i.e. swapping moving and fixed image) and compare against direct inversion.

In [ ]:
# TODO

**Task 32:** Calculate the:
- end-point-error (EPE): $\text{EPE} = \|u^{(F)} - u^{(B)}\|_2 = \sqrt{|u_x^{(F)}-u_x^{(B)}|^2 + |u_y^{(F)}-u_y^{(B)}|^2}$
- end-angulation-error (EAE): $\text{EAE} = \arg(u^{(F)}, u^{(B)})$

between the backward flow fields (direct inversion or inverse registration) and the forward flow fields. Plot the EPE and EAE as flow fields: $\text{EPE} \cdot \exp{(1i \cdot \text{EAE})}$ to have a color-coded error map (using `plot_flow()`).

*Note:* EPE and EAE can be calculated between any two flows to investigate their similarity/dissimilarity.

In [ ]:
flow = np.stack([ux, uy], -1)
flowB = np.stack([uBx, uBy], -1)
flowB_direct = np.stack([uBx_direct, uBy_direct], -1)

def epe(flow1, flow2):
  return np.sqrt(np.sum(np.power(np.abs(flow1 - flow2),2), axis=-1))

def eae(flow1, flow2):
  return np.real(np.arccos((1 + np.sum(flow1 * flow2, axis=-1)) / (np.sqrt(1 + np.sum(np.power(flow1, 2), axis=-1)) * np.sqrt(1 + np.sum(np.power(flow2, 2), axis=-1))))) * 180 / np.pi

EPE = # TODO
EPE_direct = # TODO

EAE = # TODO
EAE_direct = # TODO

flowtest = EPE * np.exp(1j * EAE)
testx = np.real(flowtest)
testy = np.imag(flowtest)
plot_flow(np.stack([testx, testy], -1), title='Isomorphism: inverted registration')
flowtest = EPE_direct * np.exp(1j * EAE_direct)
testx = np.real(flowtest)
testy = np.imag(flowtest)
plot_flow(np.stack([testx, testy], -1), title='Isomorphism: direct inversion')

**Task 33:** Check if the flows are isomorph by warping a random image. Plot the respective errors for the backward flows (inverse registration and direct inversion).

In [ ]:
# TODO

Alternatively for an affine motion, we can also analytically calculate the motion field from the affine motion matrix. We provide the function `get_transform()` to return the flow fields:

```
def get_transform(img, p):
    # img      input image to be transformed
    # p        affine transformation parameters
    #          3D (rank(img) == 3): t_x, t_y, t_z, \phi [°], \theta [°], \psi [°], G_{xy}, G_{xz}, G_{yz}, S_x, S_y, S_z
    #          2D (rank(img) == 2): t_x, t_y, \phi [°], G_{xy}, S_x, S_y
    # return   deformation field
```

**Task 34:** You may rerun the previous tasks using the analytically derived motion field and compare it against the one obtained from registration.

In [ ]:
# TODO

### B-Spline
We will now perform a B-Spline based registration on the same pair of motion images as above.

**Task 35:** Perform a cubic B-Spline registration, aligning the moving image to the fixed image. Please use a Mean-Squared-Error metric, gradient descent optimizer, pyramidal multi-resolution registration with B-Spline mesh grid sizes of $[10, 6, 2, 1]$.

*Hint:* You may use [SimpleITK](https://simpleitk.readthedocs.io/en/master/link_ImageRegistrationMethodBSpline1_docs.html) for this.

In [ ]:
import SimpleITK as sitk

# TODO
# registered flows are stored in ITK flow object "outTx"

outFlow = sitk.TransformToDisplacementField(outTx, 
                                  sitk.sitkVectorFloat64,
                                  fixed.GetSize(),
                                  fixed.GetOrigin(),
                                  fixed.GetSpacing(),
                                  fixed.GetDirection())

outFlow_np = sitk.GetArrayFromImage(outFlow)

resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(fixed)
resampler.SetInterpolator(sitk.sitkLinear)
resampler.SetDefaultPixelValue(1)
resampler.SetTransform(outTx)
warped = resampler.Execute(moving)
warped_np = sitk.GetArrayFromImage(warped)

# composing in sitk
simg1 = sitk.Cast(sitk.RescaleIntensity(fixed), sitk.sitkUInt8)
simg2 = sitk.Cast(sitk.RescaleIntensity(warped), sitk.sitkUInt8)
cimg = sitk.Compose(simg1, simg2, simg1 // 2.0 + simg2 // 2.0)
cimg_np = sitk.GetArrayFromImage(cimg)

# plotting the results
plot_flow(outFlow_np)
plot([img_fixed, img_moving, warped_np, np.abs(warped_np - img_fixed)], title='fixed | moving | warped | error')

**Task 36:** Obtain the backward motion field by [displacement field inversion](https://simpleitk.org/doxygen/latest/html/classitk_1_1simple_1_1InverseDisplacementFieldImageFilter.html) and compare it against a registration with swapped moving and fixed image. Plot the flow fields and the warped images.

In [ ]:
# flow field inversion
R = sitk.InverseDisplacementFieldImageFilter()
R.SetReferenceImage(moving)
outFlowB = R.Execute(outFlow)
outTxB = sitk.DisplacementFieldTransform(outFlowB)
outFlowB = sitk.TransformToDisplacementField(outTxB, 
                                  sitk.sitkVectorFloat64,
                                  fixed.GetSize(),
                                  fixed.GetOrigin(),
                                  fixed.GetSpacing(),
                                  fixed.GetDirection())
outFlowB_np = sitk.GetArrayFromImage(outFlowB)

resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(moving)
resampler.SetInterpolator(sitk.sitkLinear)
resampler.SetDefaultPixelValue(1)
resampler.SetTransform(outTxB)
warpedB = resampler.Execute(fixed)
warpedB_np = sitk.GetArrayFromImage(warpedB)

# inverse registration
# TODO
# registered flows are stored in ITK object "outTxBreg"

outFlowBreg = sitk.TransformToDisplacementField(outTxBreg, 
                                  sitk.sitkVectorFloat64,
                                  fixed.GetSize(),
                                  fixed.GetOrigin(),
                                  fixed.GetSpacing(),
                                  fixed.GetDirection())

outFlowBreg_np = sitk.GetArrayFromImage(outFlowBreg)

resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(moving)
resampler.SetInterpolator(sitk.sitkLinear)
resampler.SetDefaultPixelValue(1)
resampler.SetTransform(outTxBreg)
warpedBreg = resampler.Execute(fixed)
warpedBreg_np = sitk.GetArrayFromImage(warpedBreg)

# plotting the results
plot_flow(outFlowB_np, title='inversion')
plot([img_fixed, img_moving, warpedB_np, np.abs(warpedB_np - img_moving)], title='inversion: fixed | moving | warped | error')

plot_flow(outFlowBreg_np, title='registration')
plot([img_fixed, img_moving, warpedBreg_np, np.abs(warpedBreg_np - img_moving)], title='registration: fixed | moving | warped | error')


Did you spot any difference? They are most likely neglectable, but computational demand is higher for a swapped registration than inverting the flow field.

**Task 37:** Evaluate the isomorphism of the flow fields: $u^{(F)}(u^{(B)}(r)) = r$.

In [ ]:
# TODO

**Task 38:** Change the metric to a Mattes Mutual Information metric and compare the registration with the previous one.

In [ ]:
import SimpleITK as sitk

# TODO
# registered flow is stored in ITK object "outTx"

outFlow = sitk.TransformToDisplacementField(outTx, 
                                  sitk.sitkVectorFloat64,
                                  fixed.GetSize(),
                                  fixed.GetOrigin(),
                                  fixed.GetSpacing(),
                                  fixed.GetDirection())

outFlow_mi_np = sitk.GetArrayFromImage(outFlow)

resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(fixed)
resampler.SetInterpolator(sitk.sitkLinear)
resampler.SetDefaultPixelValue(1)
resampler.SetTransform(outTx)
warped = resampler.Execute(moving)
warped_mi_np = sitk.GetArrayFromImage(warped)

# plotting the results
plot_flow(outFlow_np)
plot([img_fixed, img_moving, warped_mi_np, np.abs(warped_mi_np - img_fixed)], title='fixed | moving | warped | error')

In [ ]:
# comparison between MSE and MI metric
plot_flow([outFlow_np, outFlow_mi_np, np.abs(outFlow_np - outFlow_mi_np)], title=['MSE', 'MI', 'diff: MSE <> MI'])

You have most likely observed some incorrect motion estimates at the image boundaries. These are occuring in case you have too many degrees of freedom in your motion model (e.g. B-Spline) when applying to an affine motion (6 degrees of freedom in 2D).

**Task 39:** Adjust the used transformation that better suits your underlying motion with which you simulated your image pair `img_fixed` and `img_moving`.

*Hint:* You can find an overview of the available transformations [here](https://simpleitk.readthedocs.io/en/master/registrationOverview.html).

In [ ]:
import SimpleITK as sitk

# TODO

outFlow = sitk.TransformToDisplacementField(outTx, 
                                  sitk.sitkVectorFloat64,
                                  fixed.GetSize(),
                                  fixed.GetOrigin(),
                                  fixed.GetSpacing(),
                                  fixed.GetDirection())

outFlow_trans_np = sitk.GetArrayFromImage(outFlow)

resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(fixed)
resampler.SetInterpolator(sitk.sitkLinear)
resampler.SetDefaultPixelValue(1)
resampler.SetTransform(outTx)
warped = resampler.Execute(moving)
warped_trans_np = sitk.GetArrayFromImage(warped)

# plotting the results
plot_flow(outFlow_trans_np)
plot([img_fixed, img_moving, warped_trans_np, np.abs(warped_trans_np - img_fixed)], title='fixed | moving | warped | error')

**Task 40:** Compare the flows obtained from the optical flow with the SimpleITK flows (B-Spline and other transformation as set in **Tasks 24/27, 35, 38** and **39**).

In [ ]:
plot_flow([outFlow_np, outFlow_mi_np, outFlow_trans_np, flows], title=['B-Spline MSE', 'B-Spline MI', 'Translational MSE', 'Optical Flow'])

## Deep learning-based image registration
In recent years, deep learning solutions have also been proposed to address the image registration. A short (and by far complete) overview of deep learning image registrations for (single-modality) MRI is provided here:
- [LAPNet](https://ieeexplore.ieee.org/document/9478906)
- [CARMEN](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6677286/)
- [Voxelmorph](https://ieeexplore.ieee.org/document/8633930)
- [FlowNet](https://arxiv.org/abs/1504.06852)
- [RAFT](https://arxiv.org/abs/2003.12039)
- [MRAFT](https://link.springer.com/chapter/10.1007/978-3-030-88552-6_2)
- [MoCo-MoDL](https://onlinelibrary.wiley.com/doi/10.1002/mrm.28851)
- [GRN](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9095964/)
- [DDR](https://www.sciencedirect.com/science/article/abs/pii/S0167865517301708)
- [DIRNet](https://link.springer.com/chapter/10.1007/978-3-319-67558-9_24)
- [AIRNet](https://arxiv.org/abs/1810.02583)
- [BIRNet](https://www.sciencedirect.com/science/article/abs/pii/S1361841519300805)
- [CVAE](https://arxiv.org/abs/1804.07172)
- [FCN](https://arxiv.org/abs/1801.04012)
- [DeformableReg](https://openreview.net/forum?id=HkmkmW2jM)
- [ICNet](https://arxiv.org/abs/1809.03443)
- ...



We will employ in this example the pre-trained Voxelmorph architecture - as we do not have the time and computing ressources to properly train a neural network from scratch here 😉 But we will dive into a simple exemplary training of an image registration network.

Download and extract the example data and pre-trained networks of Voxelmorph [here](https://surfer.nmr.mgh.harvard.edu/pub/data/voxelmorph/tutorial_data.tar.gz).


In [ ]:
!wget https://surfer.nmr.mgh.harvard.edu/pub/data/voxelmorph/tutorial_data.tar.gz -O data.tar.gz
!tar -xzvf data.tar.gz

**Task 41:** Register the moving and the fixed image using the pre-trained Voxelmorph architecture `brain_2d_smooth.h5`. You frist need to specify the network architecture and hyperparameters as detailed here:


```
import voxelmorph as vxm
nb_features = [
    [32, 32, 32, 32],         # encoder features
    [32, 32, 32, 32, 32, 16]  # decoder features
]
vol_shape = tuple(np.shape(img_fixed))
vxm_model = vxm.networks.VxmDense(vol_shape, nb_features, int_steps=0)

losses = ['mse', vxm.losses.Grad('l2').loss]
loss_weights = [1, 0.01]

vxm_model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4), loss=losses, loss_weights=loss_weights)

vxm_model.load_weights('brain_2d_smooth.h5')
```

The input to the network is defined as `inp = [img_moving, img_fixed]` and the prediction can be carried out with `out = vxm_model.predict(inp)` which returns a list containing the warped image and the motion field. Plot the warped image (using `plot()`) and the motion field (using `plot_flow()`).


*Hint:* Do not forget to add a batch and channel dimension (additional first and last dimension) to the inputs.

In [ ]:
# TODO

In [ ]:
plot([img_fixed, img_moving, np.squeeze(val_pred[0], (0, -1))], title='fixed | moving | warped')
plot_flow(np.squeeze(val_pred[1], 0))

**Task 42:** Perform the above registration using the pre-trained network `brain_2d_no_smooth.h5`.

In [ ]:
# TODO

plot([img_fixed, img_moving, np.squeeze(val_pred[0], (0, -1))], title='fixed | moving | warped')
plot_flow(np.squeeze(val_pred[1], 0))

Most likely you have obtained in both cases a quite artistic looking warped image together with a smooth motion field. The reason for this is, that the pre-trained architectures were trained to match sagittal slices between different subjects (inter-subject registration) and different slices from the same subject to enable an atlas-style mapping. 

Since we want to register the same moving slices onto each other, we need to retrain the architecture. We will use the axial slices of the two validation volumes in `subj1.npz` and `subj2.npz` provided with the Voxelmorph example data. The subsqeuent steps follow the [Voxelmorph tutorial](http://tutorial.voxelmorph.net/) loosely which can be used as guidance or inspiration.


We first need to rotate the images into axial orientation (`np.rot90(..., 1)`) and subsequently zeropad the images to a size of $X \times Y \times \text{slices} = 256 \times 256 \times 224$ (`zpad(..., (256, 256, 224))`). Move the axial slices now in the batch dimension (first dimension).

**Task 43:** Prepare the training data taken from the `subj1.npz` and `subj2.npz` as described above.

In [ ]:
# prepare data from 3D val_volumes
# TODO

**Task 44:** Implement a data generator for neural network training that inputs to the network the `[moving_image, fixed_image]` in which the `moving_image` is obtained by an affine transformation (using `transform_img()`) of the `fixed_image` with randomly selected affine transformation parameters `p`. Decide for a suitable range of transformation parameters `p`.

*Hint:* You may refer to the function `vxm_data_generator()` in the [Voxelmorph tutorial](http://tutorial.voxelmorph.net/) for guidance.

In [ ]:
# TODO

train_generator = vxm_data_generator_new(traindata, batch_size=8)

**Task 45:** Now train the Voxelmorph architecture using your prepared training data generator: `vxm_model.fit_generator(train_generator, epochs=nb_epochs, steps_per_epoch=steps_per_epoch, verbose=2)`. You may decide for the amount of epochs to train `nb_epochs` and the `steps_per_epoch` ($\leq 448$). Depending on your selection, training may take up to several minutes.

In [ ]:
# TODO

hist = vxm_model.fit_generator(train_generator, epochs=20, steps_per_epoch=200, verbose=2)

**Task 46:** Register the images using your trained network. Plot the warped image and the obtained flow.

*Hint:* `model.predict()` provides you the warped image and the flow field.

In [ ]:
# TODO

Were you able to train a good image registration network? A few points to consider when training your registration network: 
- the complexity of your motion (rigid, affine, elastic)
- availability of enough diverse training data
- network architecture parametrization (kernel sizes, feature maps/channels, ...)
- hyperparameters (loss, learning rate, ...)

**Task 47:** You can play around now by changing some of the above points to see if you are able to obtain a better result. 

# 3. Motion-compensated image reconstruction
<a name="MotionCompensated"></a>
In this part, we will embed the estimated motion in the image reconstruction task. 

Before diving into the motion-compensated reconstruction task, we will shortly recap an iterative SENSE image reconstruction.

The goal is to recover the clean image $x$, which is obtained by (undersampled) k-space data $y$ and corrupted by additive Gaussian white noise $n$,
$$ y = Ax + n. $$
The rawdata $y$ was aquired with multiple receive coils. The linear operator $A$ denotes the mapping from image space to k-space.

Consequently we can define and implement the multi-coil forward operator $A$ in `mriForwardOp(image, mask, smaps)` mapping from `image` to k-space and the adjoint operator $A^*$ in `mriAdjointOp(kspace, mask, smaps)` mapping from `kspace` to image domain. `smaps` describes the coil sensitivity maps, and `mask` depicts the sampling trajectory.

We consider now the following minimization problem:

$$ \min_x  E(x,y) = \min_x \frac{1}{2} \Vert Ax - y \Vert_2^2 .$$

While in image denoising we are still able to compute a closed-form solution for this problem, this is not feasible for the task of MRI reconstruction anymore. We instead use first-order optimization methods and solve this by Gradient Descent:
$$ x^{t+1} = x^{t} - \alpha \nabla_x E(x,y) $$
$$ x^{t+1} = x^{t} - \alpha A^* (Ax^t - y) $$


**Suggested Readings:**

Pruessmann et al. [SENSE: Sensitivity encoding for fast MRI](https://onlinelibrary.wiley.com/doi/abs/10.1002/%28SICI%291522-2594%28199911%2942%3A5%3C952%3A%3AAID-MRM16%3E3.0.CO%3B2-S) Magnetic Resonance in Medicine, 43(5):952-962, 1999.

## iterative SENSE

Below is the implementation of the iterative SENSE reconstruction that follows the above formalism. The interface can be used to reconstruction Cartesian and non-Cartesian data - only determined by the definition of the respective forward and adjoint operators.

The Cartesian operators are given by:
- numpy: `mriForwardOp()` and `mriAdjointOp()` in `utils/mri.py`
- Tensorflow: `MulticoilForwardOp` and `MulticoilAdjointOp` in `merlintf.keras.layers.mri`. 

The non-Cartesian operators are defined as:
- numpy: `GPUNUFFTOp.forward()` and `GPUNUFFTOp.adjoint()` in class `GPUNUFFTOp` of `utils/mri.py`
- Tensorflow: `GPUNUFFTFwd` and `GPUNUFFTAdj` in `utils/mri.py`.

The iterative SENSE reconstruction `iterativeSENSE()` in `utisl/mri.py` is automatically executed on GPU (if available and optox is used) and has the following interface:

```
def iterativeSENSE(kspace, smap=None, mask=None, noisy=None, dcf=None, flow=None,
            fwdop=MulticoilForwardOp, adjop=MulticoilAdjointOp,
            add_batch_dim=True, max_iter=10, tol=1e-12, weight_init=1.0, weight_scale=1.0, use_optox=False):
    # kspace        raw k-space data as [X, Y, coils] which will be converted to:
    #               Cartesian + no-motion compensation: [batch, coils, X, Y] or [batch, coils, X, Y, Z] or [batch, coils, time, X, Y] or [batch, coils, time, X, Y, Z] (numpy array)
    #               Cartesian + motion-compensation / non-Cartesian + no-motion/motion-comp.: [batch, X, Y, coils]
    # smap          coil sensitivity maps with same shape as kspace (or singleton dimension for time) (numpy array)
    # mask          subsampling including/excluding soft-weights with same shape as kspace (no-motion-comp) and shape: X, Y, coils, T (motion-comp) (numpy array)
    # noisy         initialiaztion for reconstructed image, if None it is created from A^H(kspace) (numpy array)
    # dcf           density compensation function (only non-Cartesian) (numpy array)
    # flow          flow field (numpy array)
    # fwdop         forward operator A
    # adjop         adjoint operator A^H
    # add_batch_dim automatically append batch dimension for GPU execution
    # max_iter      maximum number of iterations for CG/iterative SENSE
    # tol           tolerance for stopping condition for CG/iterative SENSE
    # weight_init   initial weighting for lambda regularization parameter
    # weight_scale  scaling factor for lambda regularization parameter
    # return:       reconstructed image (numpy array)
```



In the following, we will only focus on the numpy implementation (due to RAM limitations and GPU availability), but codes for GPU implementation are also provided (Attention: The GPU implementations require further improvement as currently handovers between CPU and GPU, i.e. type conversion between numpy and Tensorflow are happening).


We will see two examples below how to perform an iterative SENSE reconstruction for a 2D Cartesian and a 2D non-Cartesian imaging.

### 2D Cartesian reconstruction

In [ ]:
from merlintf.keras.layers.mri import MulticoilForwardOp, MulticoilAdjointOp

# prepare data
kspace = fft2c(img)
kspace_espirit = kspace[:,:,np.newaxis,:]
smaps_espirit = espirit.espirit(kspace_espirit, 8, 20, 0.05, 0)
smaps = smaps_espirit[:,:,0,:,0]

# Cartesian
if os.getenv("USE_OPTOX") and os.environ["USE_OPTOX"] == "true":  # Tensorflow
  A = MulticoilForwardOp(center=True, coil_axis=-3, channel_dim_defined=False)
  AH = MulticoilAdjointOp(center=True, coil_axis=-3, channel_dim_defined=False)
  use_optox = True
else:
  A = mriForwardOp
  AH = mriAdjointOp
  use_optox = False

img_recon = iterativeSENSE(kspace, smaps, fwdop=A, adjop=AH, use_optox=use_optox)
plot(img_recon)

### 2D non-Cartesian reconstruction
<a name="2DnonCart"></a>

In [ ]:
from mri.operators import NonCartesianFFT

# prepare data
nRead = np.amax(np.shape(img)[0:2])
img_qfov = zpad(img, (nRead, nRead, np.shape(img)[2])).astype(np.complex64)
img_qfov = img_qfov[:,:,np.newaxis,:]
kspace_qfov = fft2c(img_qfov)
smaps_rad_espirit = espirit.espirit(kspace_qfov, 8, 20, 0.05, 0)
smaps_rad = smaps_rad_espirit[:,:,0,:,0]
csm = np.transpose(smaps_rad, (2, 0, 1))
kpos, dcf = prepare_radial(acc=1, nRead=nRead)

# non-Cartesian
if os.getenv("USE_OPTOX") and os.environ["USE_OPTOX"] == "true":  # Tensorflow
  A = GPUNUFFTFwd(nRead, kpos, csm, dcf)
  AH = GPUNUFFTAdj(nRead, kpos, csm, dcf)
  #kspace_radial = np.squeeze(A(numpy2tensor(rss(img_qfov[:,:,0,:]), add_batch_dim=True, add_channel_dim=False)).numpy())
  kspace_radial = A.op.op(rss(img_qfov[:,:,0,:]))
  use_optox = True
else:
  nufft = GPUNUFFTOp(kpos, csm, dcf, nRead)
  A = nufft.forward
  AH = nufft.adjoint
  kspace_radial = nufft.forward(rss(img_qfov[:,:,0,:]))
  use_optox = False

img_recon_rad = iterativeSENSE(kspace_radial, csm, kpos, dcf=dcf, fwdop=A, adjop=AH, weight_scale=0.1, max_iter=2, use_optox=use_optox)
plot(img_recon_rad)

## Motion-compensated Batchelor-type reconstruction

Affine motion has a well-understood representation in k-space, however generalized elastic motion does not. An approach to correct for this type of motion is to consider the forward model of an acquisition corrupted by motion and solve the corresponding reconstruction problem [1]:

$$ \hat{x} = \arg\min_x \frac{1}{2} \Vert Ex - y \Vert_2^2 $$

where the forward model is given by:

$$ E = \sum_n A_n F C U_n $$

Under this representation, $A_n$ are the k-space sampling matrices for each motion state $n$, $F$ is the Fourier transform, $C$ are the coil sensitivities and $U_n$ are the motion fields for each motion state. The motion fields $U_n$ may be considered as sparse motion matrices where each row contains the interpolation weights $w_i^j$ associated with a given displacement. If nearest neighbour interpolation is considered, then $U_n$ become permutation matrices, i.e. only one interpolant per row and with value $w_i^j=1$. If linear interpolation is considered (as it is in this example implementation), then each row contains four interpolants (in 2D), such that $ \sum_j w_i^j = 1 $.

![](https://github.com/lab-midas/ismrm-moco-workshop/blob/master/doc/sparse_motion_mat.png?raw=true)

For a motion-compensated Batchelor-type reconstruction, we first need a simple representation of the motion as a sparse matrix.

This is implemented in `get_sparse_motion_matrix()` (converts dense flow field to sparse motion matrix) and `apply_sparse_motion()` (warps an image via matrix-vector multiplication) from `utils/motioncomp.py`. In the Cartesian case, these functions are invoked from the forward `BatchForwardOp()` $E$ and adjoint `BatchAdjointOp()` $E^H$ operators which are handed over to the iterative SENSE reconstruction `iterativeSENSE()`.

The forward and adjoint operators that take into account the motion model are defined as `BatchForwardOp()` and `BatchAdjointOp()` in `utils/mri.py`.

<br>

[1] Batchelor PG, Atkinson D, Irarrazaval P, Hill DL, Hajnal J, Larkman D. Matrix description of general motion correction applied to multishot images. Magnetic Resonance in Medicine: An Official Journal of the International Society for Magnetic Resonance in Medicine. 2005 Nov;54(5):1273-80.

**Task 48:** Apply an affine transformation to the coil-combined image (`img_cc = mriAdjointOp(kspace, np.ones_like(kspace), smaps`) and plot the motion-free and motion-affected image side-by-side (using `plot()`).

*Hint:* Please see **Task 1** for more information.

In [ ]:
kspace = fft2c(img)
kspace_espirit = kspace[:,:,np.newaxis,:]
smaps_espirit = espirit.espirit(kspace_espirit, 8, 20, 0.05, 0)
smaps = smaps_espirit[:,:,0,:,0]
smaps = smaps / rss(smaps)[:,:,np.newaxis]  # normalize coil sensitivities

# TODO

**Task 49:** Simulate a motion-corrupted acquisition such that every 2nd phase-encoding step is affected by an abrupt (and time-constant) affine motion. You can use the function `simulate_motion()` in `utils/motionsim.py`. Plot the motion-free and motion-affected scans.

*Hint:* Please see **Task 5** for more information.

In [ ]:
p = [-10, 30, 45, 0.1, 1, 1]
mask_motion = np.zeros(np.shape(kspace)[1])
mask_motion[::2] = 1
p = mask_motion[:, np.newaxis] * np.asarray(p)[np.newaxis, :]

# TODO

**Task 50:** Simulate a translational motion-affected scan (every 2nd phase-encoding line is affected by motion) using `simulate_motion()`.

In [ ]:
p = # TODO
mask_motion = np.zeros(np.shape(kspace)[1])
mask_motion[::2] = 1
p = mask_motion[:, np.newaxis] * np.asarray(p)[np.newaxis, :]

kspace_motion, mask_motion = simulate_motion(img_cc, smaps, np.ones_like(kspace), p)
img_motion = mriAdjointOp(kspace_motion, np.ones_like(kspace), smaps)

plot([img_cc, img_motion])

Before we can perform a motion-compensated image reconstruction, we need the deformation fields which are used in the forward `BatchForwardOp()` and adjoint operators `BatchAdjointOp()`.

We can either perform an image registration between the motion-free and motion-affected scan, please see **Tasks 23** to **47** in [2. Motion estimation / image registration](#MotionEstimation), or since we are applying an explicit (and known) affine motion to the image, we can analytically calculate the deformation field for it.

**Task 51:** Write the functions `get_affine_matrix()` which gives you the affine motion matrix for the motion parameters (see [Motion artifact appearance](#MotionArtifactAppearance)) and plug this into `get_deformation_field_from_affine()` which applies the affine motion matrix at each voxel location of the image to return the deformation field of shape $X \times Y \times 2$. 

*Attention:* For `get_deformation_field_from_affine()` (and also in the Matlab code), the deformation field is usually overlaid with a grid which is used in the creation of the sparse matrix. For this, a grid is created via `np.mgrid` to which the affine motion matrix from `get_affine_matrix()` is multiplied. However, the Python code does not use this grid in the creation of the sparse matrix. We therefore provide here functions that add/remove the mesh from the flow field.



In [ ]:
def get_affine_matrix(img, p):
  if len(np.shape(img)) > 2:
    raise "Only 2D processing implemented"
  x, y = np.shape(img)
  x /= 2
  y /= 2

  # translation from origin to point (x,y)
  P1 = # TODO
  # translation back to origin from point (x,y)
  P2 = # TODO
  
  # translation
  T = # TODO

  # rotation
  radians = -np.pi * np.asarray(p[2]) / 180.
  R = # TODO

  # shearing
  G = # TODO
  
  # scaling
  S = # TODO

  # affine matrix
  return np.linalg.multi_dot([P2, G, S, R, P1]) + T

In [ ]:
def get_deformation_field_from_affine(img, affine_mat):
  height, width = np.shape(img)
  gridY, gridX = np.mgrid[1:width+1, 1:height+1]
  positions = np.concatenate((np.transpose(gridX.flatten()[:, np.newaxis], (1, 0)), np.transpose(gridY.flatten()[:, np.newaxis], (1, 0)), np.ones((1, img.size))), axis=0)  # with overlaid grid
  #positions = np.concatenate((np.ones((1, img.size)), np.ones((1, img.size)), np.ones((1, img.size))), axis=0)
  u = affine_mat @ positions
  u = np.transpose(np.reshape(np.transpose(u[0:2, :], (1, 0)), (height, width, 2)), (1, 0, 2))
  return u

def add_mesh_to_def(u):
  height, width, nd = np.shape(u)
  gridY, gridX = np.mgrid[1:width+1, 1:height+1]
  q = np.zeros(np.shape(u))
  q[:,:,0] = u[:,:,0] + gridY
  q[:,:,1] = u[:,:,1] + gridX
  return q

def remove_mesh_from_def(u):
  height, width, nd = np.shape(u)
  gridY, gridX = np.mgrid[1:width+1, 1:height+1]
  q = np.zeros(np.shape(u))
  q[:,:,0] = u[:,:,0] - gridY
  q[:,:,1] = u[:,:,1] - gridX
  return q  


**Task 52:** Obtain the deformation field via `get_flow()` in `utils/motionsim.py` for a translational displacement $t_x=10$ and plot the flow field using `plot_flow()`. Compare the warped image using the obtained flow field with the directly transformed image from `transform_img()`.

*Hint:* Please see **Task 1** for more information on transforming an image, and **Task 25** for more information on flow plotting. Do not forget to scale the images to a common range before comparing them.<br/>
The function `get_flow(img, p)` is a wrapper around `get_affine_matrix()`, `get_deformation_field_from_affine()` and `remove_mesh_from_def()`. 

In [ ]:
# TODO

#### Cartesian imaging

**Task 53:** Apply the forward operator `BatchForwardOp()` to the fully-sampled motion-free image using the flow field $u$ from **Task 52** to obtain a motion-corrupted acquisition. Subsequently apply the adjoint operator `BatchAdjointOp()` on this motion-corrupted data to obtain the zero-filled motion-corrected reconstruction. Compare this against the non-motion-corrected zero-filled reconstruction (using `mriAdjointOp()`). Simulate that every 2nd phase-encoding line is affected by motion. Plot the images.

*Hint:* Since we need a motion field for all time frames, we also require the identity flow field for the first time point which we can stack into a rank 4 matrix: $X \times Y \times 2 \ (u_x, u_y) \times 2 \ (\text{Time})$ <br/>
In a similar fashion we need to stack the masks so that they have a shape: $X \times Y \times \text{Channels} \times \text{Time}$.<br/>
Please use `get_flow()` to obtain the flow field from the affine motion parameters and `get_sparse_motion_matrix()` in `utils/motioncomp.py` to obtain the sparse motion matrices. A stack of vertical sparse tensors 

```
from scipy.sparse import vstack
smm = vstack([smm0, smm1, ...])
```

is used in the Batchelor forward and adjoint operators.


In [ ]:
mask_fs = np.ones_like(kspace)
p = [-20, 40, 0, 0, 1, 1]
u = get_flow(img_cc, p)
u_id = get_flow(img_cc, [0, 0, 0, 0, 1, 1])
flow = np.stack([u_id, u], -1)
mask_motion1 = np.zeros_like(kspace)
mask_motion1[::2] = 1
mask_motion2 = np.ones_like(kspace)
mask_motion2 -= mask_motion1
masks_motion = np.stack([mask_motion1, mask_motion2], -1)
masks = np.stack([mask_fs, mask_fs], -1)

smm0 = get_sparse_motion_matrix(flow[:,:,:,0])
smm1 = get_sparse_motion_matrix(flow[:,:,:,1])

from scipy.sparse import vstack
smm = vstack([smm0, smm1])


In [ ]:
# TODO

Residual artefacts can be observed. This is because just applying the adjoint operator leads to a naive inversion, which corresponds to an image-based motion correction approach (i.e. reconstruct separate motion-resolved aliased images and then warp them to a common reference). This only works if the motion and the (undersampled) Fourier transform for each motion state commute, which is generally not the case.

Before going to an iterative SENSE reconstruction, we will solve the linear inverse problem by a simple gradient descent algorithm: compute the local gradient and take a small step in the direction that minimizes it. It iteratively solves for the solution.

**Attention**: The iterative SENSE reconstruction requires some time. The gradient descent provides a faster approximate solution.

In [ ]:
# Gradient descent solver for linear inverse problem

def grad_descent_batch(kspace, masks_motion, smaps, smm, max_it):

  Ny, Nx, Nc = np.shape(kspace)

  #Right hand side
  m = BatchAdjointOp(kspace_motion, masks_motion, smaps, smm)
  ExEHxm = BatchAdjointOp(BatchForwardOp(m, masks_motion, smaps, smm), masks_motion, smaps, smm)

  # initial residual
  r = m - ExEHxm

  rho = np.zeros((Ny,Nx,max_it+1)) + 1j * np.zeros((Ny,Nx,max_it+1))

  # 0-th iteration, i.e., the "zero-filled" recon
  rho[:,:,0] = m

  for it in range(max_it):
      
      # Compute alpha
      aux = BatchAdjointOp(BatchForwardOp(r, masks_motion, smaps, smm), masks_motion, smaps, smm)
      top = np.sum(np.transpose(np.matrix.flatten(np.conj(r))) * np.matrix.flatten(r))
      bot = np.sum(np.transpose(np.matrix.flatten(np.conj(r))) * np.matrix.flatten(aux))
      alpha = (top / bot) 
      
      # Update solution and residual
      m = m + alpha*r
      r = r - alpha*BatchAdjointOp(BatchForwardOp(r, masks_motion, smaps, smm), masks_motion, smaps, smm)
      
      # Save output
      rho[:,:,it+1] = m

  return rho

In [ ]:
# run batch recon iteratively
recon_its = grad_descent_batch(kspace_motion, masks_motion, smaps, smm, 3)

In [ ]:
# Now we can see how aliasing artefacts are completely removed when the reconstruction is properly solved 
print(np.shape(recon_its))
plot(recon_its, title='gradient descent iterations')

**Task 54:** Perform a motion-compensated image reconstruction in `iterativeSENSE()` using the ingredients from **Task 52** and **Task 53**, and a non-motion compensated image reconstruction. Compare the reconstructed images with the zero-filled reconstructions.

*Hint:* You need to use the functions `BatchelorFwd` and `BatchelorAdj` inside the iterative SENSE which are Tensorflow layers wrapping around the `BatchForwardOp()` and `BatchAdjointOp()`. The iterative SENSE requires the non-sparse motion flows in order to wrap them into tensors.

In [ ]:
# gradient descent

# TODO

In [ ]:
# iterative SENSE

# TODO

**Task 55:** Repeat the **Tasks 52** to **54** for a rotational motion, i.e. perform a motion-compensated image reconstruction on rotational motion-affected data. 

In [ ]:
# Fully-sampled sampling masks
mask_fs = np.ones_like(kspace)

# using meshless flow
p = # TODO
u = get_flow(img_cc, p)
u_id = get_flow(img_cc, [0, 0, 0, 0, 1, 1])
flow = np.stack([u_id, u], -1)
smm0 = get_sparse_motion_matrix(flow[:,:,:,0])
smm1 = get_sparse_motion_matrix(flow[:,:,:,1])
from scipy.sparse import vstack
smm = vstack([smm0, smm1])

In [ ]:
# gradient descent

# Residual aliasing artefacts present with the naive inversion, but these are 
# corrected with the iterative solution

kspace_motion = # TODO
img_motion_corr_zf = # TODO
img_no_motion_corr_zf = # TODO 

recon_its = # TODO

plot([img_cc, img_no_motion_corr_zf,img_motion_corr_zf], title='motion-free | zero-filled no motion compensation | zero-filled motion compensation')
plot(recon_its, title='motion-compensated recon: gradient descent iterations')


In [ ]:
# iterative SENSE
A = BatchelorFwd()
AH = BatchelorAdj()
img_motion_corr_recon = # TODO

plot([img_cc, img_no_motion_corr_zf,img_motion_corr_zf, img_motion_corr_recon], title='motion-free | zero-filled no motion compensation | zero-filled motion compensation | motion-comp. recon')

**Task 56:** Now we want to examine the impact of the coil sensitivity maps on a motion-compensated reconstruction for a rotational motion (as in **Task 55**). Therefore, perform a motion-compensated reconstruction with and without coil sensitivity maps. Plot and compare the images.

In [ ]:
# gradient descent

# Considerable artefacts visible for recon without coil sensitivies. This is the actual
# image with the highest data consistency. It has removed all of the motion, however it is 
# riddled with aliasing artefacts due to the k-space gaps opened by the rotational motion.
recon_its_no_csm = # TODO 

plot(recon_its, title='recon w/ csm')
plot(recon_its_no_csm, title='recon w/o csm')

In [ ]:
# iterative SENSE
# with csm
A = BatchelorFwd()
AH = BatchelorAdj()

# without csm
img_motion_corr_recon_no_csm = # TODO

plot([img_motion_corr_recon, img_motion_corr_recon_no_csm], title='with CSM | without CSM')

**Task 57:** Perform a motion-compensated image reconstruction for rotational and translational motion in a (regular and random) undersampled Cartesian acquisition. Each motion state has a different undersampling mask (but same acceleration factor).

*Hint:* Please see **Task 11** for creating an undersampling mask.

In [ ]:
p = [0, 10, 30, 0, 1, 1]
u = get_flow(img_cc, p)
u_id = get_flow(img_cc, [0, 0, 0, 0, 1, 1])
flow = np.stack([u_id, u], -1)
smm0 = get_sparse_motion_matrix(flow[:,:,:,0])
smm1 = get_sparse_motion_matrix(flow[:,:,:,1])
from scipy.sparse import vstack
smm = vstack([smm0, smm1])

mask_fs = np.ones_like(kspace)
sampling_mask1 = np.zeros_like(kspace)
sampling_mask1[:,1::3,:] = 1
sampling_mask2 = np.zeros_like(kspace)
sampling_mask2[:,2::3,:] = 1

masks = np.stack([sampling_mask1, sampling_mask2], -1)

In [ ]:
# gradient descent

# More challenging case, with undersampling and rotational motion. Coil sensitivities
# help, but residual aliasing remains

# TODO

In [ ]:
# iterative SENSE

# TODO

**Task 58:** Perform a motion-compensated image reconstruction for an affine motion ($S_x=0.9$) on regular undersampled Cartesian data with acceleration factor 2. The k-space is additionally corrupted by additive white Gaussian noise. Coil sensitivity maps are available.

In [ ]:
p = # TODO
u = get_flow(img_cc, p)
u_id = get_flow(img_cc, [0, 0, 0, 0, 1, 1])
flow = np.stack([u_id, u], -1)
smm0 = get_sparse_motion_matrix(flow[:,:,:,0])
smm1 = get_sparse_motion_matrix(flow[:,:,:,1])
from scipy.sparse import vstack
smm = vstack([smm0, smm1])

sampling_mask = generate_mask(R=2, nPE=np.shape(img_cc)[1], nFE=np.shape(img_cc)[0], mode='regular')
masks = np.stack([np.tile(sampling_mask, (1, 1, np.shape(smaps)[-1])), np.tile(sampling_mask, (1, 1, np.shape(smaps)[-1]))], -1)

# motion-affected k-space
kspace_motion = BatchForwardOp(img_cc, masks, smaps, smm)

# add noise to k-space
# TODO


In [ ]:
# gradient descent

# Reconstruct undersampled and noisy data corrupted by scaling motion. Residual 
# aliasing and noise amplification is visible in the reconstructed images.

# TODO

In [ ]:
# iterative SENSE

# TODO

**Task 59:** We want to see now what happens if we do not have access to an accurate motion model, i.e. if the image registration was inaccurate. For this purpose, we will adjust the affine motion parameters by an error term (e.g. 10% error), so that the actual obtained deformation field is not exactly reflecting the underlying applied motion. Perform a motion-compensated image reconstruction for an affine motion on regular undersampled Cartesian data. Additionally the k-space is affected by additive white Gaussian noise (refer to **Task 58**). Coil sensitivity maps are available.

In [ ]:
# sampling
sampling_mask = generate_mask(R=2, nPE=np.shape(img_cc)[1], nFE=np.shape(img_cc)[0], mode='regular')
masks = np.stack([np.tile(sampling_mask, (1, 1, np.shape(smaps)[-1])), np.tile(sampling_mask, (1, 1, np.shape(smaps)[-1]))], -1)

# using meshless u
p = [0, 0, 0, 0, 0.9, 1]
u = get_flow(img_cc, p)
u_id = get_flow(img_cc, [0, 0, 0, 0, 1, 1])
flow = np.stack([u_id, u], -1)
smm0 = get_sparse_motion_matrix(flow[:,:,:,0])
smm1 = get_sparse_motion_matrix(flow[:,:,:,1])
from scipy.sparse import vstack
smm = vstack([smm0, smm1])

# motion-affected k-space
kspace_motion = BatchForwardOp(img_cc, masks, smaps, smm)

# add noise to k-space
# TODO

# add error on motion field
p_error = p
# TODO 10% error

ue = get_flow(img_cc, p_error)
smm1e = get_sparse_motion_matrix(ue)

smme = vstack([smm0, smm1e])


In [ ]:
# gradient descent

# Reconstruct undersampled and noisy data corrupted by scaling motion with
# errors in the motion model. Errors in the motion model propagate into scaling motion
# artefacts in the reconstructed images.

# TODO

In [ ]:
# iterative SENSE

# TODO

**Task 60:** In a next step we will regularize the reconstruction by providing a prior to help with the challenging case of motion-compensated image reconstruction in undersampled Cartesian data with k-space noise. The setting is otherwise similar to **Task 59**.

In [ ]:
# A simple version of a regularized gradient descent for problems of the type
# ||Ex-k||_2 + lambda ||x-x_reg||_2, which is equivalent to solving the problem
# Fx = b, where F = E*E^H + I*lambda and b = E^H*k + lambda*x_reg

def reg_grad_descent_batch(kspace, masks_motion, smaps, smm, max_it,x_reg,lambda_reg):

  Ny, Nx, Nc = np.shape(kspace)

  #Right hand side
  m = BatchAdjointOp(kspace_motion, masks_motion, smaps, smm) + lambda_reg*x_reg
  ExEHxm = BatchAdjointOp(BatchForwardOp(m, masks_motion, smaps, smm), masks_motion, smaps, smm) + lambda_reg*m

  # initial residual
  r = m - ExEHxm

  rho = np.zeros((Ny,Nx,max_it+1)) + 1j * np.zeros((Ny,Nx,max_it+1))

  # 0-th iteration, i.e., the "zero-filled" recon
  rho[:,:,0] = m

  for it in range(max_it):
      
      # Compute alpha
      aux = BatchAdjointOp(BatchForwardOp(r, masks_motion, smaps, smm), masks_motion, smaps, smm) + lambda_reg*r
      top = np.sum(np.transpose(np.matrix.flatten(np.conj(r))) * np.matrix.flatten(r))
      bot = np.sum(np.transpose(np.matrix.flatten(np.conj(r))) * np.matrix.flatten(aux))
      alpha = (top / bot) 
      
      # Update solution and residual
      m = m + alpha*r
      r = r - alpha*BatchAdjointOp(BatchForwardOp(r, masks_motion, smaps, smm), masks_motion, smaps, smm) + lambda_reg*r
      
      # Save output
      rho[:,:,it+1] = m

  return rho


In [ ]:
from scipy.ndimage import gaussian_filter

# sampling
sampling_mask = generate_mask(R=2, nPE=np.shape(img_cc)[1], nFE=np.shape(img_cc)[0], mode='regular')
masks = np.stack([np.tile(sampling_mask, (1, 1, np.shape(smaps)[-1])), np.tile(sampling_mask, (1, 1, np.shape(smaps)[-1]))], -1)

# using get_def_field and meshless
p = [0, 0, 0, 0, 0.9, 1]
u = get_flow(img_cc, p)
u_id = get_flow(img_cc, [0, 0, 0, 0, 1, 1])
flow = np.stack([u_id, u], -1)
smm0 = get_sparse_motion_matrix(flow[:,:,:,0])
smm1 = get_sparse_motion_matrix(flow[:,:,:,1])
from scipy.sparse import vstack
smm = vstack([smm0, smm1])

# motion-affected k-space
kspace_motion = BatchForwardOp(img_cc, masks, smaps, smm)

# add noise to k-space
# TODO

# add error on motion field
p_error = p
# TODO 10% error

ue = get_flow(img_cc, p_error)
smm1e = get_sparse_motion_matrix(ue)

smme = vstack([smm0, smm1e])

img_prior = # TODO


In [ ]:
# gradient descent

# Reconstruct undersampled and noisy data corrupted by scaling motion with
# errors in the motion model, but now regularized by a low resolution prior.
# Some suppression of noise and aliasing is achieved, but superior performance
# is achieved with a regularized conjugate gradient

# TODO

In [ ]:
# iterative SENSE

# TODO

**Task 61:** Perform a similar motion-compensated image reconstruction as in **Task 60**, but now having three motion states. 

In [ ]:
from scipy.ndimage import gaussian_filter

p = # TODO
u = get_flow(img_cc, p)

p2 = # TODO
u2 = get_flow(img_cc, p2)

u_id = get_flow(img_cc, [0, 0, 0, 0, 1, 1])

flow = np.stack([u_id, u, u2], -1)

smm0 = get_sparse_motion_matrix(flow[:,:,:,0])
smm1 = get_sparse_motion_matrix(flow[:,:,:,1])
smm2 = get_sparse_motion_matrix(flow[:,:,:,2])
from scipy.sparse import vstack
smm = vstack([smm0, smm1, smm2])

# sampling
sampling_mask = generate_mask(R=2, nPE=np.shape(img_cc)[1], nFE=np.shape(img_cc)[0], mode='regular')
masks = np.stack([np.tile(sampling_mask, (1, 1, np.shape(smaps)[-1])), 
                  np.tile(sampling_mask, (1, 1, np.shape(smaps)[-1])),
                  np.tile(sampling_mask, (1, 1, np.shape(smaps)[-1]))], -1)


kspace_motion = BatchForwardOp(img_cc, masks, smaps, smm)

# add noise to k-space
# TODO


In [ ]:
# add error on motion field
p_error = # TODO
u_error = get_flow(img_cc, p_error)

p2_error = # TODO
u2_error = get_flow(img_cc, p2_error)

flow_error = np.stack([u_id, u_error, u2_error], -1)

smm1e = get_sparse_motion_matrix(u_error)
smm2e = get_sparse_motion_matrix(u2_error)

smme = vstack([smm0, smm1e, smm2e])

# image prior
img_prior = # TODO

In [ ]:
# gradient descent

# Reconstruct undersampled and noisy data corrupted by scaling motion with
# errors in the motion model and regularized by a low resolution prior but with
# 3 motion states (and less motion amplitude per state).
# Some suppression of noise and aliasing is achieved, but superior performance
# is achieved with a regularized conjugate gradient

# TODO

In [ ]:
# iterative SENSE

# TODO

**Task 62:** Repeat the above **Task 61** with changing sampling pattern per motion state (acceleration factor and/or sampled points), motion errors on motion fields, number of motion states or changing additive noise. 

#### Non-Cartesian imaging
**Attention:** If you are running this in Google Colab, you may not have enough RAM to execute the motion-compensated reconstruction together with GPUNUFFT. You need to run this locally, upgrade to a paid tier in Google Colab or have a look at the Matlab code which showcases these examples.

**Task 63:** Perform a motion-compensated image reconstruction for an affine motion on fully-sampled non-Cartesian data. The respective forward and adjoint operators are given in `utils/mri.py` as `BatchGPUNUFFTForwardOp()` and `BatchGPUNUFFTAdjointOp()`. Inside the iterative SENSE reconstruction, you need to use for Tensorflow: `BatchelorGPUNUFFTFwd` and `BatchelorGPUNUFFTAdj` (Tensorflow wrappers similar to the Cartesian case around the previously mentioned motion-compensated GPUNUFFT operators); for numpy: `BatchGPUNUFFTForwardOp()` and `BatchGPUNUFFTAdjointOp()`.

*Hint:* Please refer to the [above](#2DnonCart) example for non-Cartesian iterative SENSE. 

In [ ]:
# prepare radial image
nRead = np.amax(np.shape(img)[0:2])
img_qfov = zpad(img, (nRead, nRead, np.shape(img)[2])).astype(np.complex64)
img_qfov = img_qfov[:,:,np.newaxis,:]
kspace_qfov = fft2c(img_qfov)
smaps_rad_espirit = espirit.espirit(kspace_qfov, 8, 20, 0.05, 0)
smaps_rad = smaps_rad_espirit[:,:,0,:,0]
csm = np.transpose(smaps_rad, (2, 0, 1))
img_rad = rss(img_qfov[:,:,0,:])

# affine motion
p = [15, 10, 30, 0.1, 1, 1]
u = get_transform(img_rad, p)
u_id_rad = get_transform(img_rad, [0, 0, 0, 0, 1, 1])
flow = np.stack([u_id_rad, u], -1)
# TODO sparse flows
kpos, dcf = prepare_radial(acc=1, nRead=nRead)

# split kpos and dcf into motion states
kpos1 = kpos[::2,:]
dcf1 = dcf[::2,:]
kpos2 = kpos[1::2,:]
dcf2 = dcf[1::2,:]
kpos_all = np.stack([kpos1, kpos2], -1)
dcf_all = np.stack([dcf1, dcf2], -1)

A = BatchelorGPUNUFFTFwd(nRead, kpos_all, csm, dcf_all)
AH = BatchelorGPUNUFFTAdj(nRead, kpos_all, csm, dcf_all)
kspace_motion_rad = A.op(img_rad, kpos_all, csm, dcf_all, flow)

# no motion compensation
# TODO

# motion-compensated
# TODO

**Task 64:** Perform a motion-compensated image reconstruction for an affine motion on accelerated non-Cartesian data.

In [ ]:
p = # TODO
img_rad = rss(img_qfov[:,:,0,:])
u = get_transform(img_rad, p)
u_id_rad = get_transform(img_rad, [0, 0, 0, 0, 1, 1])
flow = np.stack([u_id_rad, u], -1)
# TODO sparse flows
kpos, dcf = prepare_radial(acc=4, nRead=nRead)

# split kpos and dcf into motion states
kpos1 = kpos[::2,:]
dcf1 = dcf[::2,:]
kpos2 = kpos[1::2,:]
dcf2 = dcf[1::2,:]
kpos_all = np.stack([kpos1, kpos2], -1)
dcf_all = np.stack([dcf1, dcf2], -1)

A = BatchelorGPUNUFFTFwd(nRead, kpos_all, csm, dcf_all)
AH = BatchelorGPUNUFFTAdj(nRead, kpos_all, csm, dcf_all)
kspace_motion_rad = A.op(img_rad, kpos_all, csm, dcf_all, flow)

# TODO

**Task 65:** Examine again the impact of using the coil sensitivity maps (with and without) for non-Cartesian imaging and compare to the reconstructions in **Task 64**.

In [ ]:
# TODO

**Task 66:** In a similar fashion, we want to examine the impact of a motion field error (i.e. inaccurate image registration) for the fully-sampled and undersampled non-Cartesian imaging. Perform a motion-compensated image reconstruction for an affine motion on regular undersampled Cartesian data. Additionally the k-space is affected by additive white Gaussian noise. Coil sensitivity maps are available.

In [ ]:
p = [15, 10, 30, 0.1, 1, 1]
img_rad = rss(img_qfov[:,:,0,:])
u = get_transform(img_rad, p)
u_id_rad = get_transform(img_rad, [0, 0, 0, 0, 1, 1])
flow = np.stack([u_id_rad, u], -1)
# TODO sparse flows
kpos, dcf = prepare_radial(acc=1, nRead=nRead)

# split kpos and dcf into motion states
kpos1 = kpos[::2,:]
dcf1 = dcf[::2,:]
kpos2 = kpos[1::2,:]
dcf2 = dcf[1::2,:]
kpos_all = np.stack([kpos1, kpos2], -1)
dcf_all = np.stack([dcf1, dcf2], -1)

A = BatchelorGPUNUFFTFwd(nRead, kpos_all, csm, dcf_all)
AH = BatchelorGPUNUFFTAdj(nRead, kpos_all, csm, dcf_all)
kspace_motion_rad = A.op(img_rad, kpos_all, csm, dcf_all, flow)

# add noise to k-space
# TODO

# add error on motion field
p_error = # TODO 10% error
u_error = get_transform(img_rad, p_error)
flow_error = np.stack([u_id_rad, u_error], -1)

# TODO

**Task 67:** Provide the same image prior for the motion-compensated image reconstruction on undersampled non-Cartesian data. The setting is otherwise similar to **Task 66**.

In [ ]:
p = # TODO
img_rad = rss(img_qfov[:,:,0,:])
u = get_transform(img_rad, p)
u_id_rad = get_transform(img_rad, [0, 0, 0, 0, 1, 1])
flow = np.stack([u_id_rad, u], -1)
kpos, dcf = prepare_radial(acc=4, nRead=nRead)

# split kpos and dcf into motion states
kpos1 = kpos[::2,:]
dcf1 = dcf[::2,:]
kpos2 = kpos[1::2,:]
dcf2 = dcf[1::2,:]
kpos_all = np.stack([kpos1, kpos2], -1)
dcf_all = np.stack([dcf1, dcf2], -1)

A = BatchelorGPUNUFFTFwd(nRead, kpos_all, csm, dcf_all)
AH = BatchelorGPUNUFFTAdj(nRead, kpos_all, csm, dcf_all)
kspace_motion_rad = A(img_rad, kpos_all, csm, dcf_all, flow)

# add noise to k-space
# TODO

# add error on motion field
p_error = # TODO 10% error
u_error = get_transform(img_rad, p_error)
flow_error = np.stack([u_id_rad, u_error], -1)

img_prior = # TODO

# TODO

**Task 68:** Perform a similar motion-compensated image reconstruction as in **Task 67**, but now having three motion states.

In [ ]:
p = # TODO
img_rad = rss(img_qfov[:,:,0,:])
u = get_transform(img_rad, p)
p2 = # TODO
u2 = get_transform(img_rad, p2)
u_id_rad = get_transform(img_rad, [0, 0, 0, 0, 1, 1])
flow = np.stack([u_id_rad, u, u2], -1)
# TODO sparse flows
kpos, dcf = prepare_radial(acc=2, nRead=nRead)

# split kpos and dcf into motion states
kpos1 = kpos[::3,:]
dcf1 = dcf[::3,:]
kpos2 = kpos[1::3,:]
dcf2 = dcf[1::3,:]
kpos3 = kpos[2::3,:]
dcf3 = dcf[2::3,:]
kpos_all = np.stack([kpos1, kpos2, kpos3], -1)
dcf_all = np.stack([dcf1, dcf2, dcf3], -1)

A = BatchelorGPUNUFFTFwd(nRead, kpos_all, csm, dcf_all)
AH = BatchelorGPUNUFFTAdj(nRead, kpos_all, csm, dcf_all)
kspace_motion_rad = A.op(img_rad, kpos_all, csm, dcf_all, flow)

# add noise to k-space
# TODO

# add error on motion field
p_error = # TODO 10% error
u_error = get_transform(img_rad, p_error)
p2_error = p2 # TODO 10% error
u2_error = get_transform(img_rad, p2_error)
flow_error = np.stack([u_id_rad, u_error, u2_error], -1)

img_prior = # TODO 

# TODO

**Task 69:** Repeat the above **Task 68** with changing sampling pattern per motion state (acceleration factor and/or sampled points), motion errors on motion fields, number of motion states or changing additive noise. 

**Task 70:** Repeat the **Tasks 53** to **69** using a different dataset, e.g. cardiac CINE.

### Further exercises 
To be performed on numerical and/or in-vivo data:
1. Evaluate the performance of both reconstructions in the presence of noise, different degrees of undersampling, model errors (coils and/or motion), etc.

2. Implement a motion corrected reconstruction using nearest neighbour or cubic spline interpolations (current implementation uses linear). Evaluate the performance of the recon, particularly in the presence of large scalings. <br/>
Notice how the motion matrix reduces to a permutation matrix in the case of nearest neighbour interpolation.

3. Inspect the convergence of the motion corrected reconstruction without normalizing the interpolants when the tranpose motion is applied (i.e. set the "motion_norm" variable inside "mtimes.m" to identity). Experiment with different interpolation strategies, rotations, scalings and shearings.

4. Combine both itSENSE recon and the motion correction recon with compressed sensing and evalute their performances as in "1.".

5. Evaluate the performance of all the previous exercises in terms of SNR, SSIM, RMSE, etc

6. Simulate a case with through-plane motion and observe the performance of a (in-plane) motion corrected reconstruction.

7. Evaluate the g-factor of the motion corrected reconstruction for different types of motion and sampling.

8. Motion correction can be combined with soft-weighting to alleviate the inherent noise amplification. With soft-weighting, the sampling matrices are real-valued [0,1] and each k-space data can partially belong to multiple motion states. Implement such a soft-weighted motion corrected reconstruction and evaluate its' performance.

9. The current implementation assumes the object is acquired during a steady state. Contrast resolved reconstructions can be easily solved using low rank subspace forward models. Combine a low rank constrained reconstruction with motion correction to enable contrast resolved motion correction.

10. Plenty of other parameters can go into the forward model. For example, B0 and B1 can vary with motion. Implement a variant of the motion corrected reconstruction that accounts for B0 and B1 variations due to motion.
